# Hyperbolic Learning in Action: Practice

In this notebook, we are going to train, evaluate, and compare three Convolutional Neural Networks (CNNs):

1. an ordinary, fully Euclidean one;
2. one with the last layer in hyperbolic space;
3. a fully hyperbolic network.

We will use:

- the CIFAR-10 and CIFAR-100 datasets, whereas the first is chosen for its simplicity and the second because it exhibits *hierarchical* structure;
- the hyperbolic learning library `HypLL` for the hyperbolic layers, due to its ease of use.

We will visualize data representations in the Euclidean and hyperbolic space.

## Setup

Start by adding the project's root to the Python path for custom functions.

In [ ]:
import os
import sys
sys.path.append(os.path.join(os.getcwd(), os.pardir))

Set the `torch` device and seeds for reproducibility.

In [ ]:
import torch
from src.utils.torch_utils import get_available_device, set_seeds

device = torch.device(get_available_device())
set_seeds(42)

Get the datasets.

Since this is a demonstration, and it does not use hyperparameter tuning, it is ok to work only with one split for training and one for evaluation, i.e. testing.

In [ ]:
import torchvision

transform = torchvision.transforms.Compose(
    [
        torchvision.transforms.ToTensor(),
        torchvision.transforms.Normalize(
            mean=(0.5, 0.5, 0.5),
            std=(0.5, 0.5, 0.5),
        ),
    ]
)
train_dataset = torchvision.datasets.CIFAR10(
    root="data", train=True, download=True, transform=transform
)
test_dataset = torchvision.datasets.CIFAR10(
    root="data", train=False, download=True, transform=transform
)

classes = train_dataset.classes
assert test_dataset.classes == classes
num_classes = len(classes)
print(f"Classes in the dataset: {classes}")

Prepare the data loaders.

The batch size and the number of workers may be adjusted as needed.

In [ ]:
batch_size = 128
num_workers = 0

train_dataloader = torch.utils.data.DataLoader(
    train_dataset, batch_size=batch_size, shuffle=True, num_workers=num_workers
)
test_dataloader = torch.utils.data.DataLoader(
    test_dataset, batch_size=batch_size, shuffle=False, num_workers=num_workers
)

## Euclidean Network

Start with a simple Euclidean convolutional network.

To compare with hyperbolic networks without too much pain:

- it has no batch normalization nor skip connections;
- fully connected layers are used at the end instead of e.g. global pooling;
- no transfer learning is used.

In [ ]:
from typing import Sequence, Tuple
from torch.nn import Conv2d, Flatten, Linear, MaxPool2d, ReLU, Sequential


def make_euclidean_backbone(
    in_channels: int = 3,
    conv_channels: Sequence[int] = tuple(),
    fc_channels: Sequence[int] = tuple(),
    conv_kernel_size: int = 3,
    pool_kernel_size: int = 2,
    pool_stride: int = 2,
    image_size: Tuple[int, int] = (32, 32),
    last_activation: bool = False,
) -> Tuple[Sequential, int]:
    all_conv_channels = (in_channels, *conv_channels)
    pool = MaxPool2d(kernel_size=pool_kernel_size, stride=pool_stride)
    activation = ReLU()
    current_image_size = torch.tensor(image_size)
    layers = []
    for i in range(len(conv_channels)):
        layers.append(
            Conv2d(
                in_channels=all_conv_channels[i],
                out_channels=all_conv_channels[i + 1],
                kernel_size=conv_kernel_size,
            )
        )
        current_image_size -= conv_kernel_size - 1
        layers.append(activation)
        layers.append(pool)
        current_image_size //= pool_stride
    layers.append(Flatten())
    all_fc_channels = (all_conv_channels[-1] * current_image_size.prod(), *fc_channels)
    for i in range(len(fc_channels)):
        layers.append(
            Linear(in_features=all_fc_channels[i], out_features=all_fc_channels[i + 1])
        )
        if i + 1 < len(fc_channels) or last_activation:
            layers.append(activation)
    return Sequential(*layers), all_fc_channels[-1]


def make_euclidean_net(
    out_channels: int = 1,
    *args,
    **kwargs,
) -> Sequential:
    backbone, backbone_channels = make_euclidean_backbone(*args, **kwargs)
    head = Linear(in_features=backbone_channels, out_features=out_channels)
    return Sequential(backbone, head)
